In [60]:
# from utils import process_data
# from data_objects.utterance import Utterance
import os, yaml, sys, random, torchaudio
import pyworld as pw
sys.path.insert(1, '/homes/bdoc3/my_utils')
from audio.worldvocoder import code_harmonic, sp_to_mfsc
from my_os import recursive_file_retrieval

In [61]:
import numpy as np
import math, librosa, warnings

# UTTERANCE CLASS

class Utterance:
    def __init__(self, frames_fpath, wave_fpath, config, feat_params):
        self.frames_fpath = frames_fpath
        self.wave_fpath = wave_fpath
        self.config = config
        self.feat_params = feat_params

    def get_chunk(self, frames, n_frames, start=None):

        if frames.shape[0] > n_frames:
            if start == None:
                start = np.random.randint(0, frames.shape[0] - n_frames)
        else:
#             print(f'frames.shape[0] {frames.shape[0]}, n_frames {n_frames}')
            start = 0
            pad_size = math.ceil(n_frames - frames.shape[0]/2)
            if frames.ndim == 1:
                pad_vec = np.full((pad_size), np.min(frames))
            else:
                pad_vec = np.full((pad_size, frames.shape[1]), np.min(frames))
            frames = np.concatenate((pad_vec, frames, pad_vec))
            
        end = start + n_frames
#         print('start', start)
        return frames[start:end], (start, end)

# get features, either from audio or precomputed npy arrays.
    def get_frames(self, n_frames, start=None):

        if self.config.use_audio:
            y, _ = sf.read(self.frames_fpath)
            samps_per_frame = (self.feat_params['frame_dur_ms']/1000) * self.feat_params['sr']
            required_size =  int(samps_per_frame * n_frames)
            if y.shape[0] < 1:
                # '+2' at end is for f0_estimation vector
                frames = np.zeros((n_frames, (self.feat_params['num_feats']+self.feat_params['num_aper_feats']+2)))
                start_end = (0, required_size)
            else:
                counter = 0
                looper = True
                while looper:
                    if counter > 10:
                        raise Exception(f'Could not find vocal segments after randomly selecting 10 segments of length {n_frame}.')
                    try:
                        if start == None:
                            y_chunk, start_end = self.get_chunk(y, required_size)
                        else:
                            y_chunk, start_end = self.get_chunk(y, required_size, start)
                        frames = process_data(y_chunk.astype('double'), self.feat_params, self.config)
                        looper = False
                    except ValueError as e:
                        print(e, 'Trying another random chunk')
                        counter +=1

        else:
            frames = np.load(self.frames_fpath)
            frames, start_end = self.get_chunk(frames, n_frames)
        # print('another utterance processed')
        return frames[:n_frames], start_end

    def random_partial(self, n_frames, num_feats):
        """
        Crops the frames into a partial utterance of n_frames
        
        :param n_frames: The number of frames of the partial utterance
        :return: the partial utterance frames and a tuple indicating the start and end of the 
        partial utterance in the complete utterance.
        """
        # pdb.set_trace()

        frames, start_end = self.get_frames(n_frames)
        frames = frames[:,:num_feats]

        # frames = (frames - frames.mean()) / frames.std() # normalise from 0-1 across entire numpy
        # frames = (frames - frames.mean(axis=0)) / frames.std(axis=0) # normalise from 0-1 across features
        # pdb.set_trace()   
        return frames, start_end

    def specific_partial(self, n_frames, num_feats, start):
        """
        Crops the frames into a partial utterance of n_frames
        
        :param n_frames: The number of frames of the partial utterance
        :return: the partial utterance frames and a tuple indicating the start and end of the 
        partial utterance in the complete utterance.
        """
        # pdb.set_trace()

        frames, start_end = self.get_frames(n_frames, start)
        frames = frames[:,:num_feats]

        # frames = (frames - frames.mean()) / frames.std() # normalise from 0-1 across entire numpy
        # frames = (frames - frames.mean(axis=0)) / frames.std(axis=0) # normalise from 0-1 across features
        # pdb.set_trace()   
        return frames, start_end 

def process_data(y, feat_params, config):

    if config.use_wav2world:
        feats=pw.wav2world(y, feat_params['sr'],frame_period=feat_params['frame_dur_ms'])
        harm = feats[1]
        aper = feats[2]
        refined_f0 = feats[0]
    else:
        if config.f0_extract == 'harvest':
            f0, t_stamp = pw.harvest(y, feat_params['sr'], feat_params['fmin'], feat_params['fmax'], feat_params['frame_dur_ms'])
        elif config.f0_extract =='dio':
            f0, t_stamp = pw.dio(y, feat_params['sr'], feat_params['fmin'], feat_params['fmax'], frame_period = feat_params['frame_dur_ms'])
        refined_f0 = pw.stonemask(y, f0, t_stamp, feat_params['sr'])
        harm = pw.cheaptrick(y, refined_f0, t_stamp, feat_params['sr'], f0_floor=feat_params['fmin'])
        aper = pw.d4c(y, refined_f0, t_stamp, feat_params['sr'])
    refined_f0 = freq_to_vuv_midi(refined_f0) # <<< this can be done at training time

    # print('basic harm/aper/f0 features extracted')

    if config.dim_red_method == 'code-h':
        harm = code_harmonic(harm, feat_params['num_feats'])
        aper = code_harmonic(aper, feat_params['num_aper_feats'])
    elif config.dim_red_method == 'world':
        harm = pw.code_spectral_envelope(harm, feat_params['sr'], feat_params['num_feats'])
        aper = pw.code_aperiodicity(aper, feat_params['num_feats'])
    elif config.dim_red_method == 'chandna':
        harm = 10*np.log10(harm) # previously, using these logs was a separate optional process to 'chandna'
        aper = 10*np.log10(aper**2)
        harm = sp_to_mfsc(harm, feat_params['num_feats'], 0.45)
        aper =sp_to_mfsc(aper, 4, 0.45)
    else:
        raise Exception("The value for dim_red_method was not recognised")
    # print(f'{random.randint(0,100)}feature dims reduced')


    out_feats=np.concatenate((harm,aper,refined_f0),axis=1)

    return out_feats


def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isinf(y), lambda z: z.nonzero()[0]


def freq_to_vuv_midi(f0):
    "Convert to midi notes, with second vector displaying 1 when there's no pitch detected"
    with warnings.catch_warnings(): # warning 
        warnings.simplefilter("ignore", category=RuntimeWarning)
        notes_y = 69+12*np.log2(f0/440)
    y = notes_y
    "Nan related"
    nans, x= nan_helper(y)
    if np.all(nans) == True:
        raise ValueError('No voice pitch detected in segment')
    naners=np.isinf(y)
    y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    y=np.array(y).reshape([len(y),1])
    guy=np.array(naners).reshape([len(y),1])
    y=np.concatenate((y,guy),axis=-1)
    return y

In [62]:
data_path = '/homes/bdoc3/my_data/audio_data/deslienced_concat_DAMP'

# with open(os.path.join(data_path, 'feat_params.yaml'), 'rb') as Handle:
#     feat_params = yaml.load(Handle, Loader=yaml.FullLoader)

class Object():
    pass

config = Object()
config.use_audio = True
config.use_wav2world = True
config.f0_extract = 'harvest'
config.dim_red_method = 'chandna'
feat_params = {"use_wav2world":config.use_wav2world,
                                "f0_extract":config.f0_extract,
                                "dim_red_method":config.dim_red_method,
                                "fmin":50,
                                "fmax":1100,
                                'num_feats':40,
                                'num_aper_feats':4,
                                'frame_dur_ms':10,
                                'sr':16000,
                                'fft_size':None}
n_frames = 307
num_feats = 40

In [8]:
# issue_singers = ['1468732648']
issue_singers = ['1006811699']
# issue_singers = ['424702701']

uttrs = []
counter = 0
while counter<20:
    for i in issue_singers:
        dir_path = os.path.join(data_path, 'train', i)
        if not os.path.exists(dir_path):
            dir_path = os.path.join(data_path, 'val', i)
        _, _, files = next(os.walk(dir_path))
        for file_name in files:
            print(f'Processing {file_name}, process {counter}')
            file_path = os.path.join(dir_path, file_name)
            u = Utterance(file_path, file_path, config, feat_params)
            u_part = u.random_partial(n_frames, num_feats)
            uttrs.append(u_part)
            counter += 1


Processing 1006811699_1843697812.wav, process 0
start 1534031
start 1965938
start 846338
start 1179270
start 681702
start 137574
start 1343017
start 96237
start 1510890
start 794940
start 667331
start 1889658
start 285500
start 1509357
start 1409037
start 2888039
start 43960
start 1068158
start 863627
start 27331
start 693680
start 2848947
start 844222
start 453365


In [ ]:
# Test with specific example

ex = '/homes/bdoc3/my_data/audio_data/deslienced_concat_DAMP/train/424702701/424702701_1734334861.wav'
start = 1425394
u = Utterance(ex, ex, config, feat_params)
u.random_partial(n_frames, num_feats)

In [21]:
all_dirs, files = recursive_file_retrieval(data_path)
wav_files = [f for f in files if f.endswith('wav')]

# %timeit librosa.load(wav_files[random.randint(0, len(wav_files))]) #  1.57 s ± 631 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# %timeit sf.read(wav_files[random.randint(0, len(wav_files))]) #53.8 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# %timeit torchaudio.load(wav_files[random.randint(0, len(wav_files))]) #44.2 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The slowest run took 4.87 times longer than the fastest. This could mean that an intermediate result is being cached.
44.2 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
%%timeit
# with librosa IO - 1.98 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

file_path = wav_files[random.randint(0, len(wav_files))]
u = Utterance(file_path, file_path, config, feat_params)
u_part = u.random_partial(n_frames, num_feats)

y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
y.shape[0] is 16000, s is 16000
y.shape[0] is 16000, required_size is 49120
frames.shape[0] 16000, n_frames 49120
start 0
947 ms ± 45.9 ms per loo

In [57]:
file_path = wav_files[random.randint(0, len(wav_files))]
file_path = '/homes/bdoc3/my_data/audio_data/deslienced_concat_DAMP/train/374399338/374399338_1741657119.wav'
u = Utterance(file_path, file_path, config, feat_params)
u.frames_fpath
u.random_partial(307, 40)

start 664268


(array([[ -5.53413142,  -5.16761401,  -6.13489713, ..., -23.56622401,
         -24.75902111, -35.46205331],
        [ -7.44956002,  -8.1365492 ,  -7.68187535, ..., -22.74666417,
         -23.07679609, -32.91781073],
        [ -7.82786729,  -8.06915859,  -7.88852626, ..., -24.24993222,
         -24.08777826, -32.59682484],
        ...,
        [ -4.24098048,  -5.18928258,  -4.57552981, ..., -21.62389486,
         -20.34128273, -28.53943275],
        [ -4.33712282,  -5.15864007,  -4.64674877, ..., -21.21071475,
         -19.01734806, -30.59034519],
        [ -4.28971528,  -4.9698928 ,  -4.67096784, ..., -19.60396634,
         -22.53332584, -33.21100276]]),
 (664268, 713388))

frames.shape[0] 16000, n_frames 49120
start 0


(array([[-74.45549213, -74.51635275, -75.16445553, ..., -73.59117998,
         -73.65194068, -72.67943397],
        [-73.9299961 , -73.77400554, -73.17402136, ..., -72.65817129,
         -73.17156661, -73.65419276],
        [-72.97933636, -73.16329091, -73.07653586, ..., -73.91498246,
         -73.36513359, -73.14998229],
        ...,
        [-16.52725893, -16.92269527, -16.2995616 , ..., -22.22038767,
         -25.00117668, -36.94109211],
        [-17.82533619, -17.51704452, -19.22908632, ..., -22.72612031,
         -23.27058227, -32.21532981],
        [-17.97780601, -18.45214683, -17.85974323, ..., -24.0291684 ,
         -27.3111865 , -34.40743662]]),
 (0, 49120))

In [30]:
qwe, asd = sf.read('/homes/bdoc3/my_data/audio_data/deslienced_concat_DAMP/train/374399338/374399338_1741657119.wav')

In [31]:
qwe, asd

(array([-0.03210449, -0.05941772, -0.10882568, ...,  0.06515503,
         0.06286621,  0.06729126]),
 16000)

In [36]:
qwe.shape[0]

1732608